In [1]:
import time
import string
import re
import math
import nltk
nltk.download('punkt')
from collections import Counter
from nltk.stem.porter import *
from nltk.corpus import stopwords
from sklearn.metrics import f1_score, precision_score, recall_score

data = {}
new_id = -1   # current id
topic_flag = False   # topic has value or not
body_flag = False   # body has value or not
body = ''
id_list = []   # all id

begin = time.time()
filename = ["data/reut2-%03d.sgm" % r for r in range(0, 22)]
for fn in filename:
    print('preprocessing ' + fn + ' begin~~~')
    file = open(fn, 'r', encoding = 'ISO-8859-1')
    line = file.readlines()

    for l in line:
        if l.find('REUTERS TOPICS') != -1:
            if topic_flag:   # 去除前一篇有topic沒有body的部分
                del data[new_id]
                new_id = -1
                topic_flag = False
                
            dtype = re.split('[=]*["]+[ ]*', l)
            if dtype[1] == 'YES':   # 判斷REUTERS TOPICS
                new_id = dtype[9]   # new_id
                data[new_id] = {}
                data[new_id]['type'] = dtype[3]   # LEWISSPLIT => TRAIN or TEST
                
        elif l.find('<TOPICS><D>') != -1 and new_id != -1:
            topic_flag = True
            kind = re.split('<TOPICS><D>|</D><D>|</D></TOPICS>\n', l)   # 切出每個topic
            kind.pop()
            kind.pop(0)
            data[new_id]['category'] = kind   # topic category
            
        elif l.find('<BODY>') != -1 and new_id != -1:
    #         去除沒有topic label的部分
            if not topic_flag:
                del data[new_id]
                new_id = -1
                continue
                
            body_flag = True
            body = re.split('<BODY>', l)[1].strip()   # body的第一行
            continue
            
        elif l.find('</BODY>') != -1 and new_id != -1 and topic_flag:  # body的最後一行
            data[new_id]['body'] = body
            id_list.append(new_id)    # store the new id
            body_flag = False
            topic_flag = False
            new_id = -1
            body = ''

        if body_flag and topic_flag:
            body += l.strip()   # 讀取body每一行並連接在一起
end = time.time()
print('\nread data: ', end - begin, 's')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


preprocessing data/reut2-000.sgm begin~~~
preprocessing data/reut2-001.sgm begin~~~
preprocessing data/reut2-002.sgm begin~~~
preprocessing data/reut2-003.sgm begin~~~
preprocessing data/reut2-004.sgm begin~~~
preprocessing data/reut2-005.sgm begin~~~
preprocessing data/reut2-006.sgm begin~~~
preprocessing data/reut2-007.sgm begin~~~
preprocessing data/reut2-008.sgm begin~~~
preprocessing data/reut2-009.sgm begin~~~
preprocessing data/reut2-010.sgm begin~~~
preprocessing data/reut2-011.sgm begin~~~
preprocessing data/reut2-012.sgm begin~~~
preprocessing data/reut2-013.sgm begin~~~
preprocessing data/reut2-014.sgm begin~~~
preprocessing data/reut2-015.sgm begin~~~
preprocessing data/reut2-016.sgm begin~~~
preprocessing data/reut2-017.sgm begin~~~
preprocessing data/reut2-018.sgm begin~~~
preprocessing data/reut2-019.sgm begin~~~
preprocessing data/reut2-020.sgm begin~~~
preprocessing data/reut2-021.sgm begin~~~

read data:  2.396456241607666 s


In [2]:
def stem(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tf(text):
    lowers = text.lower()   # 轉小寫
    
#     remove punctuatuin
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lowers.translate(remove_punctuation_map)
    
    tokens = nltk.word_tokenize(no_punctuation)   # string to token
    
    filtered = [w for w in tokens if not w in stopwords.words('english')]   # stopword remove
    
    stemmer = PorterStemmer()   # stemming
    stemmed = stem(filtered, stemmer)
    
    count = Counter(stemmed)   # count the token
    
    return dict(count)

In [ ]:
bag = {}
train_count = 0   # 訓練集數量

begin = time.time()
file = open('data/all-topics-strings.lc.txt', 'r')   # read the topic file
line = file.readlines()

# bag initial
for l in line:
    bag[l.strip()] = {}
    bag[l.strip()]['body'] = ''   # 此topic的所有文章tf
    bag[l.strip()]['count'] = 0   # 此topic包含多少篇文章

for i in id_list:
    if data[i]['type'] == 'TRAIN':
        for t in data[i]['category']:
            bag[t]['body'] += ' ' + data[i]['body']   # 合併所有屬於此topic的文章
            bag[t]['count'] += 1
            train_count += 1
            
for key in bag.keys():
    count = 0
    if bag[key]['body'] != '':
        bag[key]['body'] = tf(bag[key]['body'])   # tf
        for word in bag[key]['body'].keys():
            count += bag[key]['body'][word]
    bag[key]['allcount'] = count   # 此topic總字數

# 若此topic沒有任何內容則將其去除，以免影響後續計算
for key in list(bag):
    if bag[key]['count'] == 0:
        del bag[key]
        
end = time.time()
print('bag construct: ', end - begin, 's')

In [ ]:
def naive_bayes(test_data):
    f_score = {}
    
    for topic in bag.keys():   # 對於每一個topic去算目前這個test的分數
        occur = 0   # 計算目前這個test是否有詞不在此topic中，0表示沒有
        score = 0
        
        for word in test_data.keys():
            if word not in bag[topic]['body'].keys():
                occur = 1
                break
        
        for word in test_data.keys():
            for tf in range(test_data[word]):
#                 calculate the probability
#                 P(B|A)P(A)
#                 由於分母皆相同，所以將其去除並不影響結果
                if word in bag[topic]['body'].keys():
                    score += math.log((bag[topic]['body'][word] + occur) / (bag[topic]['allcount'] + occur))
                else:
                    score += math.log(occur/(bag[topic]['allcount'] + occur))
                    
        if bag[topic]['count'] != 0:
            score += len(test_data) * math.log(bag[topic]['count'] / train_count)
        else:
            score = 0
        
        f_score[topic] = score   # 此test在所有topic的score
        
    return max(f_score, key=f_score.get)

In [ ]:
test_count = 0   # 測試集數目
correct = 0   # 預測成功數目
test_actually = []   # 實際結果
test_predict = []   # 預測結果

begin = time.time()
for i in id_list:
    if data[i]['type'] == 'TEST':
        data[i]['body'] = tf(data[i]['body'])   # tf
        topic = naive_bayes(data[i]['body'])   # predict topic
        test_count += 1
        
        if topic in data[i]['category']:   # predict correct
            correct += 1
            test_actually.append(topic)
        else:   # predict incorrect
            test_actually.append(data[i]['category'][0])
        test_predict.append(topic)

end = time.time()
print('testing set preprocessing and predict: ', end - begin, 's\n')

In [ ]:
p = precision_score(test_actually, test_predict, average='macro')
r = recall_score(test_actually, test_predict, average='macro')
f1 = f1_score( test_actually, test_predict, average='macro' )

print('precision: ', p)
print('recall: ', r)
print('F-measure: ', f1)
print('accuracy: ', (correct / test_count))

In [ ]:
print(train_count)